In [1]:
file_path = "C:/Azure_kinect_project/test_MS1/test_MS2/m12_4.txt"

f = open(file_path, 'r')
lines = f.readlines()

output = []
frame = []
joint = 0
for line in lines:
    joint+=1
    line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
    line = line.split(":")
    line = line[1]
    
    line = line.strip(')')
    line = line.strip('(')
    line = line.split(",")

    for i in range(3):
        line[i] = float(line[i])
    frame.append(line)
    
    if joint%32 == 0:
        output.append(frame)
        joint = 0
        frame = []
f.close() 
#print(output)

In [2]:
import numpy as np

In [3]:
### 최대각도 구하기 #1 ###
#output[frame][joint][x,y,z]

# ∠
# the joints in test limb
angleList = []
for i in range (len(output)):
    joint_1_xyz = np.array(output[i][4])  # a
    joint_2_xyz = np.array(output[i][7])  # b
    # the joints in plane
    joint_3_xyz = np.array(output[i][4])   # c
    joint_4_xyz = np.array(output[i][18])   # d

    # the vectors of above three joints.
    vector1 = joint_2_xyz - joint_1_xyz
    vector2 = joint_4_xyz - joint_3_xyz     # Fig_11_2 m09 Active straight raise
    # Vector2 = (1, 0, 0)   # Fig_12_2 m11 Trunk stability

    # the norm of vectors
    norm_vector1 = np.sqrt(vector1.dot(vector1))
    norm_vector2 = np.sqrt(vector2.dot(vector2))
    # the dot product of vectors
    dot_product = vector1.dot(vector2)
    # the radians of the angle
    angle_cos = dot_product / (norm_vector1 * norm_vector2)
    # convert to an angle value
    angle_deg = (np.arccos(angle_cos) * 180) / np.pi

    angleList.append(angle_deg)
#print(angleList)
print('angle Max : ', max(angleList))

angle Max :  99.08361426425355


In [7]:
### 최대각도 구하기 #2 ###

local_angleList = []
max_angleList = []
real_max_angleList = []
CMP = 1 #최댓값 구하기 위해 비교할 앞,뒤 각각의 frame 개수

for i in range(CMP,len(angleList)-CMP):
    for j in range(1,CMP+1):
        if (angleList[i] > angleList[i-j]) and (angleList[i] > angleList[i+j]):
            if (j == CMP):
                local_angleList.append(angleList[i])
        else:
            break

print(local_angleList)

if len(local_angleList) > 12:
    for k in range(len(local_angleList)):
        if k == 0:
            if (local_angleList[k] > local_angleList[k+1]):
                max_angleList.append(local_angleList[k])
        elif k == len(local_angleList)-1:
            if(local_angleList[k] > local_angleList[k-1]):
                max_angleList.append(local_angleList[k])
        elif (local_angleList[k] > local_angleList[k-1]) and (local_angleList[k] > local_angleList[k+1]):
            max_angleList.append(local_angleList[k])
    
    new_max_angleList = sorted(max_angleList, reverse=True)[:12]
    
    for a in max_angleList:
        for b in new_max_angleList:
            if a == b:
                real_max_angleList.append(a)

print(real_max_angleList)

[13.308325116558512, 13.897624807146821, 14.032355543969143, 13.901233447331718, 96.72372869568859, 96.80360571881167, 13.758041253143796, 95.22438317936198, 95.3202214056708, 13.169155765028563, 12.924391724078847, 12.889423070910814, 92.73295367229552, 92.26494014245856, 13.955295443990735, 14.220636702342574, 96.72469637859594, 13.527700241192255, 14.050082237128466, 14.024087808886856, 13.696396557140375, 94.97430545386398, 95.89774985250992, 13.62302854940297, 13.901302957585113, 95.1978222290265, 95.19552985285978, 95.07899801209182, 14.46674426457047, 14.464366171566821, 95.26599432590127, 94.11546026279835, 13.83230114354765, 97.11304716269314, 12.931443338500303, 13.290231062668049, 13.297124407482404, 13.048085815649145, 97.11106848566534, 97.5422297775495, 97.4692469731189, 95.00893947723895, 95.6496506770214, 13.499003199258237, 13.070028901328307, 12.5476026380814, 12.844616789009207, 97.5798882614951, 97.66661397682174, 97.5850196998917, 97.27031988026252, 13.135117084960

In [8]:
#get the data feature "x" for (max, min, mean, and variance)
import statistics as st

X_max = max(real_max_angleList)
X_min = min(real_max_angleList)
X_mean = sum(real_max_angleList)/len(real_max_angleList)
X_variance = st.pvariance(real_max_angleList)

print(X_max,'\t', X_min ,'\t' ,X_mean ,'\t' ,X_variance)

99.08361426425355 	 92.73295367229552 	 96.24984995342923 	 2.398053053716737


In [ ]:
import pandas as pd

pd.read_csv("C:/Azure_kinect_project/test_MS1/test_MS2/max,min,mean,variance.csv")